In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import sklearn
import datetime

In [2]:
df07 = pd.read_csv("Dataset_activaciones_SAMUR/activaciones_samur_2017.csv",sep=';', encoding='latin-1')
df08 = pd.read_csv("Dataset_activaciones_SAMUR/activaciones_samur_2018.csv",sep=';', encoding='latin-1')
df09 = pd.read_csv("Dataset_activaciones_SAMUR/activaciones_samur_2019.csv",sep=';', encoding='latin-1')
dataframe = df09
#Cambias el dataset por el que quieras, lo procesas con el notebook entero.

In [3]:
month_dict = {"ENERO":1,"FEBRERO":2,"MARZO":3,"ABRIL":4,"MAYO":5,"JUNIO":6,"JULIO":7,"AGOSTO":8,"SEPTIEMBRE":9,"OCTUBRE":10,"NOVIEMBRE":11,"DICIEMBRE":12} 
#Diccionario para traducir los meses a integers
year = dataframe["Año"].unique()[0] 
#Año del dataset, para incluirlo en la fecha

In [4]:
dataframe

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital
0,2019,ENERO,0:08:09,0:08:15,Intoxicación etílica,CENTRO,NaN
1,2019,ENERO,0:09:13,NaN,Violencia de genero,VALLECAS PTE.,NaN
2,2019,ENERO,0:24:12,0:24:22,Otros,CENTRO,NaN
3,2019,ENERO,0:28:55,NaN,Intoxicación etílica,CENTRO,NaN
4,2019,ENERO,0:29:11,0:34:02,"Casual: caída, etc",CENTRO,NaN
...,...,...,...,...,...,...,...
149899,2019,DICIEMBRE,23:28:26,23:35:50,Intoxicación etílica,CHAMARTIN,NaN
149900,2019,DICIEMBRE,23:32:19,23:37:39,Patología digestiva,LATINA,Central de la Defensa
149901,2019,DICIEMBRE,23:35:18,23:38:41,Intoxicación etílica,ARGANZUELA,NaN
149902,2019,DICIEMBRE,NaN,NaN,1 SVB,CENTRO,NaN


In [5]:
def striptimeHandler(x):
    if x == -1:
        return -1
    else:
        return datetime.datetime.strptime(x,"%X")
#Ligero cambio a datetime.strptime para poder utilizarlo en la columna de intervención, donde existen Nans.

In [6]:
dataframe["Hospital"].fillna("No derivado",inplace=True)
#Donde no hay hospital, el paciente no fue derivado a uno, bien porque se le asistió in situ o porque el recurso fue cancelado
dataframe["Hora Intervención"].fillna(-1,inplace=True)
#Donde no hay hora de intervención, el recurso fue cancelado. Se rellena ese valor con un -1.
dataframe.dropna(inplace=True)
#Donde no ha hora de solicitud, la salida fue rutinaria o no exixste el dato. Se quitan esas salidas. 
dataframe["Hora Solicitud"] = dataframe["Hora Solicitud"].apply(lambda x : datetime.datetime.strptime(x,"%X"))
dataframe["Hora Intervención"] = dataframe["Hora Intervención"].apply(lambda x : striptimeHandler(x))
#Se transforman las horas a datetime, para utilizarlas en comparaciones y condensar todos los datos en una fecha total 
dataframe.reindex()
#Reindexación del dataset para evitar discrepancias de índices

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital
0,2019,ENERO,1900-01-01 00:08:09,1900-01-01 00:08:15,Intoxicación etílica,CENTRO,No derivado
1,2019,ENERO,1900-01-01 00:09:13,-1,Violencia de genero,VALLECAS PTE.,No derivado
2,2019,ENERO,1900-01-01 00:24:12,1900-01-01 00:24:22,Otros,CENTRO,No derivado
3,2019,ENERO,1900-01-01 00:28:55,-1,Intoxicación etílica,CENTRO,No derivado
4,2019,ENERO,1900-01-01 00:29:11,1900-01-01 00:34:02,"Casual: caída, etc",CENTRO,No derivado
...,...,...,...,...,...,...,...
149897,2019,DICIEMBRE,1900-01-01 23:12:54,1900-01-01 23:20:13,"Casual: caída, etc",HORTALEZA,Ramón y Cajal
149898,2019,DICIEMBRE,1900-01-01 23:26:55,1900-01-01 23:31:16,Apertura de puerta,CHAMBERI,No derivado
149899,2019,DICIEMBRE,1900-01-01 23:28:26,1900-01-01 23:35:50,Intoxicación etílica,CHAMARTIN,No derivado
149900,2019,DICIEMBRE,1900-01-01 23:32:19,1900-01-01 23:37:39,Patología digestiva,LATINA,Central de la Defensa


In [7]:
dataframe["Devuelto"] = dataframe["Hora Intervención"].apply(lambda x : (x == -1))
#Se añade Devuelto, que indica si el recurso ha llegado a su destino o si se ha dado la vuelta al medio

In [8]:
# Encontrar los dias explota el orden entre las horas que existe en el dataset original.
# Las horas de las intervenciones en el dataset comienzan en 00:00:00 y aumenta hasta alguna hora cercana a las 23:59:59 y vuelven a 00:00:00.
# Eso significa que ha transcurrido un dia.
# Iterando sobre una secuencia de fechas, comparas una con la siguiente, y si es mayor la siguiente es del dia siguiente. 
#Con una variable day cuentas los dias desde el primer dia de la lista.
def encontrar_los_dias(dataframe):
    temp = dataframe["Hora Solicitud"].to_numpy()
    dias = [0]
    day = 0
    for i in range(0,len(temp)-1):
        dias.append(day)
        if(temp[i] >temp[i+1]):
            day +=1
    return dias

In [9]:
#Simple algoritmo que aplana un array de dos dimensiones a uno de una dimensión
def flatten2D(array):
    temp = []
    for i in array:
        for x in i:
            temp.append(x)
    return temp

In [10]:
#Aplicamos el algoritmo al dataset, mes por mes para evitar errores.
#Se aplana el array para poder incluirlo en el dataset
dias_desde_inicio = []
for mes in dataframe["Mes"].unique():
    dias_desde_inicio.append(encontrar_los_dias(dataframe[dataframe["Mes"] == mes]))
dias_desde_inicio = flatten2D(dias_desde_inicio)
dataframe["Dias_desde_inicio"] = dias_desde_inicio

In [11]:
#El máximo de Dias desde inicio es 30 y no 31 porque el primer dia del mes tiene un valor de 0 y no 1.
dataframe.describe()

,Año,Dias_desde_inicio
count,146780.0,146780.000000
mean,2019.0,14.682736
std,0.0,8.882540
min,2019.0,0.000000
25%,2019.0,7.000000
50%,2019.0,15.000000
75%,2019.0,22.000000
max,2019.0,30.000000


In [12]:
#Se construye la fecha de cada solicitud, incluyendo el año del dataset y el mes de la solicitud.
#Se suma uno al dia para corregir la desviación causada por encontrar el dia.
#La hora, minuto y segundo se encuentran extrayéndolas del datetime ya creado previamente.
#Un nuevo objeto datetime se crea con estos datos y se añade a ua lista.
#El proceso se aplica tanto a la solicitud como a la intervención, con la diferencia de que se evita el valor -1, ya que eso significa que esa hora no existe.
fecha_solicitud = []
fecha_intervención = []
for x in dataframe.iterrows():
    year = x[1][0]
    month = month_dict.get(x[1][1])
    day = x[1][8] + 1
    hour = x[1][2].hour
    minute = x[1][2].minute
    second = x[1][2].second
    temp = datetime.datetime(year, month,day,hour,minute,second)
    fecha_solicitud.append(temp)
for x in dataframe.iterrows():
    if x[1][7]: 
        fecha_intervención.append(-1)
    else:
        year = x[1][0]
        month = month_dict.get(x[1][1])
        day = x[1][8] + 1
        hour = x[1][3].hour
        minute = x[1][3].minute
        second = x[1][3].second
        temp = datetime.datetime(year, month,day,hour,minute,second)
        fecha_intervención.append(temp)

In [13]:
#Las listas se añaden al dataset y se eliminan las columnas redundantes.
dataframe["Solicitud"] = fecha_solicitud
dataframe["Intervención"] = fecha_intervención
dataframe.drop(["Hora Solicitud","Hora Intervención","Dias_desde_inicio"],axis=1,inplace=True)

In [14]:
#Finalmente, se encuentra el dia de la semana en aquella fecha. 0 es lunes y 6 es domingo.
dataframe["Dia de la semana"] = dataframe["Solicitud"].apply(lambda x: x.weekday())

In [15]:
#Así quedan el dataset
dataframe

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana
0,2019,ENERO,Intoxicación etílica,CENTRO,No derivado,False,2019-01-01 00:08:09,2019-01-01 00:08:15,1
1,2019,ENERO,Violencia de genero,VALLECAS PTE.,No derivado,True,2019-01-01 00:09:13,-1,1
2,2019,ENERO,Otros,CENTRO,No derivado,False,2019-01-01 00:24:12,2019-01-01 00:24:22,1
3,2019,ENERO,Intoxicación etílica,CENTRO,No derivado,True,2019-01-01 00:28:55,-1,1
4,2019,ENERO,"Casual: caída, etc",CENTRO,No derivado,False,2019-01-01 00:29:11,2019-01-01 00:34:02,1
...,...,...,...,...,...,...,...,...,...
149897,2019,DICIEMBRE,"Casual: caída, etc",HORTALEZA,Ramón y Cajal,False,2019-12-31 23:12:54,2019-12-31 23:20:13,1
149898,2019,DICIEMBRE,Apertura de puerta,CHAMBERI,No derivado,False,2019-12-31 23:26:55,2019-12-31 23:31:16,1
149899,2019,DICIEMBRE,Intoxicación etílica,CHAMARTIN,No derivado,False,2019-12-31 23:28:26,2019-12-31 23:35:50,1
149900,2019,DICIEMBRE,Patología digestiva,LATINA,Central de la Defensa,False,2019-12-31 23:32:19,2019-12-31 23:37:39,1


In [16]:
dataframe[dataframe["Mes"] == "AGOSTO"]

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana
88050,2019,AGOSTO,Agresión sin especificar,LATINA,Doce de Octubre,False,2019-08-01 00:09:35,2019-08-01 00:18:04,3
88051,2019,AGOSTO,Intoxicación etílica,CENTRO,No derivado,False,2019-08-01 00:12:22,2019-08-01 00:20:47,3
88052,2019,AGOSTO,Servicios de seguimiento de riesgos,MONCLOA,No derivado,False,2019-08-01 00:20:12,2019-08-01 00:25:09,3
88053,2019,AGOSTO,Patología anafiláctica,ARGANZUELA,No derivado,False,2019-08-01 00:38:17,2019-08-01 00:47:17,3
88054,2019,AGOSTO,Patología psicosomática,CENTRO,No derivado,False,2019-08-01 00:50:39,2019-08-01 00:55:43,3
...,...,...,...,...,...,...,...,...,...
98091,2019,AGOSTO,Intoxicación etílica,VALLECAS PTE.,INFANTA LEONOR,False,2019-08-31 23:38:15,2019-08-31 23:41:43,5
98092,2019,AGOSTO,Agresión sin especificar,CENTRO,No derivado,False,2019-08-31 23:39:02,2019-08-31 23:48:21,5
98093,2019,AGOSTO,Otros,CENTRO,No derivado,False,2019-08-31 23:40:47,2019-08-31 23:48:45,5
98094,2019,AGOSTO,Autolisis no traumática,MORATALAZ,Gregorio Marañón,False,2019-08-31 23:56:12,2019-08-31 00:07:57,5


In [17]:
#fancy_substracion se usa para poder hallar el tiempo de recorrido por las ambulancias. 
#Como hay algunas que se dan la vuelta y no tienen tiempo de llegada, la resta tiene que poder lidiar con el placeholder -1. 
#Se aplica la resta a todo el dataset para encontrar el tiempo de recorrido
def fancy_substraction(x,y):
    if x == -1 or y == -1:
        return -1
    else:
        return x-y
    
tiempo_recorrido = []
for x in dataframe.iterrows():
    tiempo_recorrido.append(fancy_substraction(x[1][7],x[1][6]))
dataframe["Tiempo de recorrido"] = tiempo_recorrido

In [18]:
dataframe[dataframe["Mes"] == "AGOSTO"]

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana,Tiempo de recorrido
88050,2019,AGOSTO,Agresión sin especificar,LATINA,Doce de Octubre,False,2019-08-01 00:09:35,2019-08-01 00:18:04,3,0 days 00:08:29
88051,2019,AGOSTO,Intoxicación etílica,CENTRO,No derivado,False,2019-08-01 00:12:22,2019-08-01 00:20:47,3,0 days 00:08:25
88052,2019,AGOSTO,Servicios de seguimiento de riesgos,MONCLOA,No derivado,False,2019-08-01 00:20:12,2019-08-01 00:25:09,3,0 days 00:04:57
88053,2019,AGOSTO,Patología anafiláctica,ARGANZUELA,No derivado,False,2019-08-01 00:38:17,2019-08-01 00:47:17,3,0 days 00:09:00
88054,2019,AGOSTO,Patología psicosomática,CENTRO,No derivado,False,2019-08-01 00:50:39,2019-08-01 00:55:43,3,0 days 00:05:04
...,...,...,...,...,...,...,...,...,...,...
98091,2019,AGOSTO,Intoxicación etílica,VALLECAS PTE.,INFANTA LEONOR,False,2019-08-31 23:38:15,2019-08-31 23:41:43,5,0 days 00:03:28
98092,2019,AGOSTO,Agresión sin especificar,CENTRO,No derivado,False,2019-08-31 23:39:02,2019-08-31 23:48:21,5,0 days 00:09:19
98093,2019,AGOSTO,Otros,CENTRO,No derivado,False,2019-08-31 23:40:47,2019-08-31 23:48:45,5,0 days 00:07:58
98094,2019,AGOSTO,Autolisis no traumática,MORATALAZ,Gregorio Marañón,False,2019-08-31 23:56:12,2019-08-31 00:07:57,5,-1 days +00:11:45


In [19]:
dataframe[dataframe["Mes"] == "DICIEMBRE"]

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana,Tiempo de recorrido
136399,2019,DICIEMBRE,Otros,CENTRO,Ramón y Cajal,False,2019-12-01 00:01:41,2019-12-01 00:02:17,6,0 days 00:00:36
136401,2019,DICIEMBRE,Patología cardiovascular,CENTRO,No derivado,False,2019-12-01 00:07:09,2019-12-01 00:11:40,6,0 days 00:04:31
136402,2019,DICIEMBRE,Patología cardiovascular,VALLECAS PTE.,No derivado,False,2019-12-01 00:09:28,2019-12-01 00:14:46,6,0 days 00:05:18
136403,2019,DICIEMBRE,Patología cardiovascular,CHAMARTIN,No derivado,False,2019-12-01 00:09:56,2019-12-01 00:18:01,6,0 days 00:08:05
136404,2019,DICIEMBRE,Patología psicosomática,VICALVARO,No derivado,True,2019-12-01 00:13:55,-1,6,-1
...,...,...,...,...,...,...,...,...,...,...
149897,2019,DICIEMBRE,"Casual: caída, etc",HORTALEZA,Ramón y Cajal,False,2019-12-31 23:12:54,2019-12-31 23:20:13,1,0 days 00:07:19
149898,2019,DICIEMBRE,Apertura de puerta,CHAMBERI,No derivado,False,2019-12-31 23:26:55,2019-12-31 23:31:16,1,0 days 00:04:21
149899,2019,DICIEMBRE,Intoxicación etílica,CHAMARTIN,No derivado,False,2019-12-31 23:28:26,2019-12-31 23:35:50,1,0 days 00:07:24
149900,2019,DICIEMBRE,Patología digestiva,LATINA,Central de la Defensa,False,2019-12-31 23:32:19,2019-12-31 23:37:39,1,0 days 00:05:20


In [20]:
#Se puede ver que para algunas salidas a altas horas de la noche el tiempo de recorrido aparece de forma incorrecta, debido a que la fecha de intervención aparece como anterior a la fecha de salida.
#Todos estos tiempos tienen los minutos y segundos de forma correcta, pero aparecen con -1 dia.
#Para corregirlo, se comprueba que los tiempos de recorrido no sean -1 dias, y se suma uno si es así con testtraveltime()
#Se aplica este tiempo de recorrido para corregir las fechas de intervención, sumando el tiempo de recorrido a la fecha de salida, que siempre existe.
def testtraveltime(x):
    if x == -1:
         return -1
    elif x.days == -1:
        return x+ datetime.timedelta(days=1)
    else:
        return x
dataframe["Tiempo de recorrido"] = dataframe["Tiempo de recorrido"].apply(lambda x: testtraveltime(x))

new_dates = []
for x in dataframe.iterrows():
    if x[1][9] == -1:
        new_dates.append(-1)
    else:
        new_dates.append(x[1][9] + x[1][6])
dataframe["Intervención"] = new_dates
dataframe

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana,Tiempo de recorrido
0,2019,ENERO,Intoxicación etílica,CENTRO,No derivado,False,2019-01-01 00:08:09,2019-01-01 00:08:15,1,0 days 00:00:06
1,2019,ENERO,Violencia de genero,VALLECAS PTE.,No derivado,True,2019-01-01 00:09:13,-1,1,-1
2,2019,ENERO,Otros,CENTRO,No derivado,False,2019-01-01 00:24:12,2019-01-01 00:24:22,1,0 days 00:00:10
3,2019,ENERO,Intoxicación etílica,CENTRO,No derivado,True,2019-01-01 00:28:55,-1,1,-1
4,2019,ENERO,"Casual: caída, etc",CENTRO,No derivado,False,2019-01-01 00:29:11,2019-01-01 00:34:02,1,0 days 00:04:51
...,...,...,...,...,...,...,...,...,...,...
149897,2019,DICIEMBRE,"Casual: caída, etc",HORTALEZA,Ramón y Cajal,False,2019-12-31 23:12:54,2019-12-31 23:20:13,1,0 days 00:07:19
149898,2019,DICIEMBRE,Apertura de puerta,CHAMBERI,No derivado,False,2019-12-31 23:26:55,2019-12-31 23:31:16,1,0 days 00:04:21
149899,2019,DICIEMBRE,Intoxicación etílica,CHAMARTIN,No derivado,False,2019-12-31 23:28:26,2019-12-31 23:35:50,1,0 days 00:07:24
149900,2019,DICIEMBRE,Patología digestiva,LATINA,Central de la Defensa,False,2019-12-31 23:32:19,2019-12-31 23:37:39,1,0 days 00:05:20


In [21]:
dataframe[dataframe["Intervención"] == -1]

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana,Tiempo de recorrido
1,2019,ENERO,Violencia de genero,VALLECAS PTE.,No derivado,True,2019-01-01 00:09:13,-1,1,-1
3,2019,ENERO,Intoxicación etílica,CENTRO,No derivado,True,2019-01-01 00:28:55,-1,1,-1
27,2019,ENERO,Heridas,MORATALAZ,No derivado,True,2019-01-01 01:45:37,-1,1,-1
29,2019,ENERO,"Casual: caída, etc",SAN BLAS,No derivado,True,2019-01-01 01:50:41,-1,1,-1
36,2019,ENERO,Accidente menos de 3 victimas,MONCLOA,No derivado,True,2019-01-01 02:00:54,-1,1,-1
...,...,...,...,...,...,...,...,...,...,...
149841,2019,DICIEMBRE,Agresión sin especificar,HORTALEZA,No derivado,True,2019-12-31 19:50:48,-1,1,-1
149847,2019,DICIEMBRE,Otros,USERA,No derivado,True,2019-12-31 20:05:29,-1,1,-1
149862,2019,DICIEMBRE,Convulsión y/o epilepsia,SALAMANCA,No derivado,True,2019-12-31 20:53:11,-1,1,-1
149869,2019,DICIEMBRE,Heridas,CARABANCHEL,No derivado,True,2019-12-31 21:06:24,-1,1,-1


In [22]:
(13119/146780)*100 #9% de las ambulancias se dan la vuelta antes de llegar a  su destino

8.937866194304402

In [23]:
#Falta varios dias en el dataset. Utilizaré el patrón existente entre dias de la semana y intoxicaciones etílicas para intentar estimar los que faltan.
#Se usa el otro notebook, SAMUR Dataset Study

In [24]:
#Para su facil manejo, se exporta a .csv
dataframe.to_csv("Dataset_SAMUR_2019.csv")